## **MINERALS OF ENERGY TRANSITION** 

**EXTRACTION, TRANSFORMATION, LOAD (ETL)**

**DISCLAIMER:** <br>
*The data used for analyzing the market was obtained from open sources, the information and insights in the present document can't be used with commercial purposes keeping each data source with their original licences.*

**SUMMARY:** <br>
TBD

**OBJECTIVE:** <br>
To extract and transform data from the origin considering the production values as main or target variable.


### 00.00. RESOURCES AND WORK ENVIRONMENT SETTING
#### 00.01. LIBRARIES AND WD

In [1]:
##-- ENVIRONMENT SETTINGS
import pandas as pd
import numpy as np
import os
import json

import sqlite3

import warnings
warnings.filterwarnings('ignore')

In [2]:
#-- Custom Libraries
main_wd = os.getcwd()[:os.getcwd().find('2023.Minerals-EDA') + len('2023.Minerals-EDA') +1]
os.chdir(main_wd)
os.chdir([x for x in json.load(open('./config/config.json',))[0]['directory'] if 'wd_custom_libraries' in x][0]['wd_custom_libraries'])    
import aux_time_series

os.chdir(main_wd)

In [3]:
##-- Work Directory  
wd_in = [x for x in json.load(open('./config/config.json',))[0]['data'] if 'raw' in x][0]['raw']
wd_out = [x for x in json.load(open('./config/config.json',))[0]['data'] if 'processed' in x][0]['processed']
csvAttr_imp = json.load(open('./config/config.json',))[0]['csvAttr_imp'][0]
csvAttr_exp = json.load(open('./config/config.json',))[0]['csvAttr_exp'][0]

#### 00.02 DATA
There are two main datasets, The first one contains the production data with its data point of international trade by producer country but both concepts are not equally completed due to coming from different sources.
The other dataset contains only the trade but for every single country in the source database. 

The mineral units of measure have been standardized for trade and production in the [ETL main process]( https://github.com/zapallo-droid-ca/2023.Minerals-ETL) (other project) where petroleum and natural gas are in tonnes of oil equivalent and the rest of minerals in tonnes.

The minerals query contains a custom variable with the dimension for what the observation or row can be utilized for, considering that trade only contains values for years after 1986 and production has older values.


In [4]:
##-- Queries
ft_minerals_q = open(main_wd + '/ETL/ft_minerals.sql').read()

##-- Connection
conn = sqlite3.connect(wd_in + '/minerals_db.db')
cursor = conn.cursor()

##-- Data
#Fact Tables
ft_minerals = pd.read_sql(ft_minerals_q, conn)

#Aux Data
dim_mineral = pd.read_sql('SELECT mineral_code , mineral_desc  FROM dim_mineral dm',conn)

#-- Closing Connection
conn.close()

del(conn, cursor, ft_minerals_q)

The main sources only have values for production or trade in those years where transactions were made, to work with time series and to get the statistical measures and values, the queries extracted data using a cross-join function between the fact tables and calendar table contained in the Data Warehouse where all the null values where imputed with “0”.

During the query process, the calculations of rolling aggregations and ratios were made to delegate the process to SQL.


### 01.00. TRANSFORMATIONS
#### 01.01. Time Series Decomposition





In [5]:
##-- Parameters
target = 'quantity_produced'

ft_minerals = ft_minerals.groupby(['mineral_code','year']).agg(target = (target,'sum')).copy().reset_index()

#--Indexing DateTime
ft_minerals.set_index(pd.to_datetime(ft_minerals['year'], format = '%Y'), inplace = True)

#--key for future joins
ft_minerals['key'] = ft_minerals['year'].astype(str) + '-' + ft_minerals['mineral_code']
#considering that there are non-unique values for the index, the frequency will be settled when is needed.

#Saving shape to final control
struct_shape = ft_minerals.shape[0]

In [6]:
##--Taking a look into the target table
ft_minerals.sort_values('target', ascending = False).head(2)

,mineral_code,year,target,key
year,,,,
2020-01-01,cm02,2020,1.862819e+09,2020-cm02
2019-01-01,cm02,2019,1.857300e+09,2019-cm02


In [7]:
##-- Decomposing the Target Series
ft_prod_tsdecomp = aux_time_series.tsDecomposition(data = ft_minerals, index_frequency = 'YS', period = 12, target = 'target', category = 'mineral_code', scale_data = False)
ft_prod_tsdecomp.head()

process ended, df_timeSeriesDecomp: (765, 6)


,date,mineral_code,level_original,residual,seasonal,trend
0,1970-01-01,cm01,9.645000e+06,0.0,-616996.840567,0.0
14,1970-01-01,cm15,1.806100e+04,0.0,-774.397859,0.0
13,1970-01-01,cm14,2.501471e+03,0.0,-27.480806,0.0
12,1970-01-01,cm13,1.064358e+03,0.0,10.074592,0.0
11,1970-01-01,cm12,1.374170e+05,0.0,-13903.330150,0.0


In [8]:
##--Adding components to main ft (fact table) 
#join key with ft_minerals
ft_prod_tsdecomp['key'] = ft_prod_tsdecomp['date'].dt.year.astype(str) + '-' + ft_prod_tsdecomp['mineral_code']

ft_minerals = ft_minerals.merge(ft_prod_tsdecomp[['key','level_original','residual','seasonal','trend']], how = 'left', on = 'key')
ft_minerals.head()

,mineral_code,year,target,key,level_original,residual,seasonal,trend
0,cm01,1970,9645000.0,1970-cm01,9645000.0,0.0,-6.169968e+05,0.0
1,cm01,1971,10266000.0,1971-cm01,10266000.0,0.0,-5.284295e+05,0.0
2,cm01,1972,11620000.0,1972-cm01,11620000.0,0.0,-1.871823e+04,0.0
3,cm01,1973,12745000.0,1973-cm01,12745000.0,0.0,-3.046605e+06,0.0
4,cm01,1974,13843000.0,1974-cm01,13843000.0,0.0,-2.035884e+06,0.0


In [9]:
##--Categories (To be able to work only with this dataset)
ft_prod_tsdecomp = ft_prod_tsdecomp.merge(dim_mineral, on = 'mineral_code', how = 'left')

#Reordering
ft_prod_tsdecomp = ft_prod_tsdecomp[['key','mineral_code','mineral_desc','date','level_original','trend','seasonal','residual']]
ft_prod_tsdecomp.head()

,key,mineral_code,mineral_desc,date,level_original,trend,seasonal,residual
0,1970-cm01,cm01,aluminium,1970-01-01,9.645000e+06,0.0,-616996.840567,0.0
1,1970-cm15,cm15,uranium,1970-01-01,1.806100e+04,0.0,-774.397859,0.0
2,1970-cm14,cm14,petroleum,1970-01-01,2.501471e+03,0.0,-27.480806,0.0
3,1970-cm13,cm13,natural gas,1970-01-01,1.064358e+03,0.0,10.074592,0.0
4,1970-cm12,cm12,molybdenum,1970-01-01,1.374170e+05,0.0,-13903.330150,0.0


### 02.00. CLUSTERING
#### 02.01. Dynamic Time Warping Similarity

The dataset contains multiple dimensions due to its categories; The countries could be taken as the subject of analysis but on the other hand, we have also values for each mineral like another category. The procedure will consider each mineral for clustering the countries.

Clustering is a classification problem and time series datasets are not compatible with clustering without reshaping. To tackle this the dataset will be reshaped getting each month ordered by columns and separating the procedure by component, so, we will clustering countries by mineral and also by component, this means that we will have x clusters by mineral on the trend component, y clusters by mineral on the seasonal component and z clusters by mineral on the residual component.

In this case, the Dynamic Time Warping (DTW) will be used as measure of similarity among the component series aiming to capture similarities with consistent shapes.

[Here is an nice explaination of the DTW Algorithm](https://www.youtube.com/watch?v=_K1OsqCicBY
)


In [10]:
##--Pivoting Dataset and getting DTW Matrix 
#- Trend Component
dtw_matrix_trend, dtw_index_trend = aux_time_series.dtw_matrix_funct(data = ft_prod_tsdecomp, index = 'mineral_code', columns = 'date', values = 'trend')

#- Seasonal Component
dtw_matrix_seaso, dtw_index_seaso = aux_time_series.dtw_matrix_funct(data = ft_prod_tsdecomp, index = 'mineral_code', columns = 'date', values = 'seasonal')

#- Residual Component
dtw_matrix_resid, dtw_index_resid = aux_time_series.dtw_matrix_funct(data = ft_prod_tsdecomp, index = 'mineral_code', columns = 'date', values = 'residual')

#### 02.02. Kmeans

To analyze and determine the optimal number of clusters three measures will be considered, the silohuette score, the calinski harabasz score and the davies bouldin score for each clustering algorithm

In [11]:
#Parameters
randomStateValue = 92
early_stop_yield = 0.1 #% of total categories without changing results
categories = ['mineral_code']


In [12]:
#- Trend Component
result_t, clusters_tests_t = aux_time_series.clustering_kmeans_multi(x = dtw_matrix_trend, 
                                                                     y = dtw_index_trend,
                                                                    categories = categories, 
                                                                    early_stop_yield = early_stop_yield, 
                                                                    randomStateValue = randomStateValue)

2.0 iters without change reached (0.1 yield), iters without change by measure -> sil: 2, cal: 0, dav: 2
process finished 



In [13]:
#- Seasonal Component
result_s, clusters_tests_s = aux_time_series.clustering_kmeans_multi(x = dtw_matrix_seaso,
                                                                     y = dtw_index_seaso,
                                                                    categories = categories, 
                                                                    early_stop_yield = early_stop_yield, 
                                                                    randomStateValue = randomStateValue)

2.0 iters without change reached (0.1 yield), iters without change by measure -> sil: 2, cal: 0, dav: 2
process finished 



In [14]:
#- Residual Component
result_r, clusters_tests_r = aux_time_series.clustering_kmeans_multi(x = dtw_matrix_resid,
                                                                     y = dtw_index_resid,
                                                                    categories = categories, 
                                                                    early_stop_yield = early_stop_yield, 
                                                                    randomStateValue = randomStateValue)

2.0 iters without change reached (0.1 yield), iters without change by measure -> sil: 2, cal: 0, dav: 2
process finished 



In [15]:
##-- Unpacking Data
clustered_data = pd.DataFrame()
selection_criteria = 'suggested_clusters'

packing_data = [('trend',result_t),('seasonal',result_s),('residual',result_r)]

for package in packing_data:
    by = package[0]
    data = package[1]    

    for key in list(data.keys()):    
        extracted_data = data[key].copy()
        extracted_data = extracted_data[['index',selection_criteria]]
        extracted_data.columns = ['index','cluster']

        extracted_data['category'] = key        
        extracted_data['by'] = by   
        #extracted_data['key_join'] = extracted_data['category'] + '-' + extracted_data['index'] 
        clustered_data = pd.concat([clustered_data,extracted_data])    

clustered_data.head()

,index,cluster,category,by
0,cm01,0,mineral_code,trend
1,cm02,1,mineral_code,trend
2,cm03,0,mineral_code,trend
3,cm04,0,mineral_code,trend
4,cm05,0,mineral_code,trend


In [16]:
##-- Pivoting Data and Creating Join Key
clustered_data = clustered_data.pivot(index = ['index','category'], columns = 'by', values = 'cluster').reset_index()
clustered_data.columns = clustered_data.columns.rename('')
clustered_data.sort_values('index', ascending = True, inplace = True)

clustered_data['key'] = clustered_data['index']
clustered_data = clustered_data[['key','trend','seasonal','residual']]
clustered_data.columns = ['key','cluster_by_trend_dtw','cluster_by_seasonal_dtw','cluster_by_residual_dtw']
clustered_data.head()

,key,cluster_by_trend_dtw,cluster_by_seasonal_dtw,cluster_by_residual_dtw
0,cm01,0,0,0
1,cm02,1,1,1
2,cm03,0,0,0
3,cm04,0,0,0
4,cm05,0,0,0


In [17]:
##--Adding components to main ft (fact table) 
#key in ft_minerals
ft_minerals.drop(columns = 'key', inplace = True)
ft_minerals['key'] = ft_minerals['mineral_code']

#join
ft_minerals = ft_minerals.merge(clustered_data, how = 'left', on = 'key')
ft_minerals.drop(columns = 'key', inplace = True)
ft_minerals.head()

,mineral_code,year,target,level_original,residual,seasonal,trend,cluster_by_trend_dtw,cluster_by_seasonal_dtw,cluster_by_residual_dtw
0,cm01,1970,9645000.0,9645000.0,0.0,-6.169968e+05,0.0,0,0,0
1,cm01,1971,10266000.0,10266000.0,0.0,-5.284295e+05,0.0,0,0,0
2,cm01,1972,11620000.0,11620000.0,0.0,-1.871823e+04,0.0,0,0,0
3,cm01,1973,12745000.0,12745000.0,0.0,-3.046605e+06,0.0,0,0,0
4,cm01,1974,13843000.0,13843000.0,0.0,-2.035884e+06,0.0,0,0,0


In [18]:
struct_shape == ft_minerals.shape[0]

True

### 03.00. Outliers Detection

Considering that we have the residual component, we can use it to determine outliers in the time series, in this case we are going to apply the Hampel Filter wihout imputations.

The outliers detection will be made for each country series.

In [19]:
data = ft_minerals.copy()
index = 'year'
target = 'residual'
category = 'mineral_code'

In [20]:
iterators = data[category].unique()

outliers = pd.DataFrame()
for iter in iterators:
    X = data[data[category] == iter][[index,target]].reset_index(drop = True).sort_values(index, ascending = True).copy()

    results = aux_time_series.hampel_filter(data = X, 
                                            index = 'year',
                                            target = 'residual',
                                            windows_size = 10, 
                                            n_sigmas = 3)

    results['key'] = results[index].astype(str) + '-' + iter

    outliers = pd.concat([outliers,results])

outliers.shape[0] == ft_minerals.shape[0]

True

In [21]:
##--Adding outliers to main ft (fact table) 
#key in ft_minerals
ft_minerals['key'] = ft_minerals[index].astype(str) + '-' + ft_minerals[category]

#join
ft_minerals = ft_minerals.merge(outliers[['key','outlier',f'imputed_{target}_values']], how = 'left', on = 'key')
ft_minerals.head()

,mineral_code,year,target,level_original,residual,seasonal,trend,cluster_by_trend_dtw,cluster_by_seasonal_dtw,cluster_by_residual_dtw,key,outlier,imputed_residual_values
0,cm01,1970,9645000.0,9645000.0,0.0,-6.169968e+05,0.0,0,0,0,1970-cm01,False,0.0
1,cm01,1971,10266000.0,10266000.0,0.0,-5.284295e+05,0.0,0,0,0,1971-cm01,False,0.0
2,cm01,1972,11620000.0,11620000.0,0.0,-1.871823e+04,0.0,0,0,0,1972-cm01,False,0.0
3,cm01,1973,12745000.0,12745000.0,0.0,-3.046605e+06,0.0,0,0,0,1973-cm01,False,0.0
4,cm01,1974,13843000.0,13843000.0,0.0,-2.035884e+06,0.0,0,0,0,1974-cm01,False,0.0


In [22]:
struct_shape == ft_minerals.shape[0]

True

### 04.00. Exporting Data

In [23]:
##--Python EDA
ft_minerals.to_csv(wd_out + '/ft_minerals_tsc.csv.gz', sep = csvAttr_exp['sep'], index = False, encoding = csvAttr_exp['encoding'])

In [24]:
##--BI Tools
#- ft_minerals
cols_to_drop = ['cluster_by_trend_dtw', 'cluster_by_seasonal_dtw', 'cluster_by_residual_dtw']
        
ft_minerals.drop(columns = cols_to_drop).to_csv(wd_out + '/BI/ft_minerals.csv.gz', sep = csvAttr_exp['sep'], index = False, encoding = csvAttr_exp['encoding'])

#- Clusters
ft_cluster = ft_minerals[['mineral_code','cluster_by_trend_dtw', 'cluster_by_seasonal_dtw',
                           'cluster_by_residual_dtw']].copy().drop_duplicates().reset_index(drop = True)

ft_cluster.to_csv(wd_out + '/BI/ft_cluster_minerals.csv.gz', sep = csvAttr_exp['sep'], index = False, encoding = csvAttr_exp['encoding'])